## Import packages

In [ ]:
import torch
import torch.nn as nn

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
# ignore warnings
import warnings ; warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import normalize
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import OLSInfluence


# Encoding
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#pickle
import pickle

In [ ]:
# 설치된 글꼴 이름 파악을 위해서!!
import matplotlib as mpl
from matplotlib import font_manager as fm
from matplotlib import rc

# 설정 파일 위치 확인
print(mpl.matplotlib_fname())
print(mpl.__file__)

# Jupyter Notebook 로드 시 같이 올라온 font 목록 확인
[f.fname for f in fm.fontManager.ttflist]	# NanumGothic.ttf 목록에서 확인 완료!!!

# NanumGothic 설정 확인
plt.rcParams['font.family'] = 'NanumGothic'
print(plt.rcParams['font.family'])		# NanumGothic 설정 제대로 먹힌 것을 확인!!!

# 마이너스(-)가 '□'로 깨지는 것을 방지하기 위하여 설정!!!
mpl.rcParams['axes.unicode_minus'] = False


In [ ]:
def summary(df):
    summary = pd.DataFrame(df.dtypes, columns=['데이터 타입'])
    summary = summary.reset_index()
    summary = summary.rename(columns={'index':'피쳐'})
    summary['결측값 개수'] = df.isnull().sum().values
    summary['고유값 개수'] = df.nunique().values
    summary['첫번째 값'] = df.loc[0].values
    summary['두번째 값'] = df.loc[1].values
    return summary

## Data Preprocessing

In [ ]:
df = pd.read_csv('1004_df.csv', low_memory=False)

In [ ]:
# 청장년층으로 필터링
df = df[df['age'].isin(['20e', '20s', '30e', '30s'])]

In [ ]:
df.date = df.date.astype('str')

In [ ]:
# 'year'와 'month' 컬럼 생성
df['year'] = df['date'].str[:2] 
df['month'] = df['date'].str[2:]  

In [ ]:
df.year= df.year.astype('int')
df.month= df.month.astype('int')
df.date = df.date.astype('int')

In [ ]:
# 2개월 전의 값으로 2개월 후의 lb08m을 설명하는 통계 모델을 만들고 예측값을 지표로 사용하고자 한다.

ease = df.query('date > 2208')[['lb08m','month','date','year','gender','age','region']].copy() 
ease['month'] = (ease['month'] - 2).astype(str).str.zfill(2)  # 한 자리수 월을 두 자리수로 만듭니다.

ease['year'] = ease['year'].astype(str)
ease['date'] = ease['year'] + ease['month']  # 'year'와 'month'를 합쳐 'date'를 만듭니다.


In [ ]:
ease['date'] = ease['date'].replace('2300', '2212')
ease = ease.drop(['year','month'],axis=1)
df.date = df.date.astype('str')

In [ ]:
Y = df.query('date=="2306"')
y = df['lb08m']

In [ ]:
df.date=df.date.astype('int')
ease.date=ease.date.astype('int')

In [ ]:
df = df.query('date<2306').drop(['lb08m','year','month'],axis=1).merge(ease, on =['date','gender','age','region'])

## Encoding

In [ ]:
# LabelEncoder를 객체로 생성
encoder = LabelEncoder()

# fit, transform 메소드를 통한 레이블 인코딩
encoder.fit(df['age'])

df["age"] = encoder.transform(df['age'])


Y["age"] = encoder.transform(Y['age'])

In [ ]:
#성별
oh = OneHotEncoder()
encoder = oh.fit_transform(df['gender'].values.reshape(-1,1)).toarray() # 인코딩 하기 전에 2차원 데이터로 변환
df_OneHot = pd.DataFrame(encoder, columns=["gender_" + str(oh.categories_[0][i]) for i in range (len(oh.categories_[0]))])

df = pd.concat([df, df_OneHot], axis=1)

# test 데이터에 transform
encoder = oh.transform(Y['gender'].values.reshape(-1, 1)).toarray()
df_test_OneHot = pd.DataFrame(encoder, columns=["gender_" + str(oh.categories_[0][i]) for i in range (len(oh.categories_[0]))])
Y = pd.concat([Y.reset_index(drop=True), df_test_OneHot], axis=1).drop(columns=['gender'])

In [ ]:
df.drop(columns = ['gender','광역시도명','구명','행정동명','시군구명'], inplace=True)
Y.drop(columns = ['광역시도명','구명','행정동명','시군구명','month','year'], inplace=True)

In [ ]:
df = df[df.notnull().all(axis=1)]
print(df.isna().sum().sum())

Y=Y[Y.notnull().all(axis=1)]
print(Y.isna().sum().sum())

In [ ]:
target=Y.lb08m
y_train=df.lb08m

In [ ]:
Y.drop(columns = ['lb08m'], inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# train 데이터에 fit_transform
X_train_scaled = pd.DataFrame(scaler.fit_transform(df.iloc[:,3:-2]), columns=df.iloc[:,3:-2].columns)

# test 데이터에 transform
X_test_scaled = pd.DataFrame(scaler.transform(Y.iloc[:,3:-2]), columns=Y.iloc[:,3:-2].columns)

print("Train Data:")
print(X_train_scaled)

print("\nTest Data:")
print(X_test_scaled)

In [ ]:
temp=pd.DataFrame(X_train_scaled)
temp_test=pd.DataFrame(X_test_scaled)

In [ ]:
temp = pd.concat([df.loc[:,['date', 'age', 'region', 'gender_F', 'gender_M']].reset_index(drop=True),temp], axis=1)

In [ ]:
temp_test =  pd.concat([Y.loc[:,['date', 'age', 'region', 'gender_F', 'gender_M']].reset_index(drop=True),temp_test], axis=1)

In [ ]:
df = temp.copy()

In [ ]:
X_test=temp_test.copy()

## 상관행렬 계산

In [ ]:
# 상관행렬 계산
correlation_matrix = df.corr().abs()

# 상관계수가 0.8 이상인 변수들을 선택
high_corr_var = np.where(correlation_matrix > 0.9)

# (i, j) 인덱스 튜플의 리스트 생성
high_corr_var = [(correlation_matrix.columns[x], correlation_matrix.columns[y]) for x, y in zip(*high_corr_var) if x != y and x < y]

# 상관계수가 높은 변수들을 제거
for var_pair in high_corr_var:
    df.drop(var_pair[1], axis=1, inplace=True)  # 두 번째 변수를 제거


In [ ]:
print(high_corr_var)

## 그룹사별 변수 추출

In [ ]:
bank = df.loc[:, ['date', 'gender_F','age', 'region'] + list(df.loc[:, 'ba01m':'be07r'].columns)]
card = df.loc[:, ['date', 'gender_F',  'age', 'region'] + list(df.loc[:, 'ca01m':'ch07r'].columns)]
stock = df.loc[:, ['date', 'gender_F', 'age', 'region'] + list(df.loc[:, 'sa01r':'sd07r'].columns)]
life = df.loc[:, ['date', 'gender_F',  'age', 'region'] + list(df.loc[:, 'la01r':'lf07r'].columns)]

## 단계선택법

In [ ]:
def stepwise_feature_selection(X, y):
    variables=X.columns.tolist()
    steps = []
    sv_per_step, adjusted_r_squared = [], [] ## 각 스텝별로 선택된 변수들, 수정된 결정계수
    selected_variables = [] ## 선택된 변수들
    
    step = 0
    while len(variables) > 0:
        remainder = list(set(variables) - set(selected_variables))
        pval = pd.Series(index=remainder) 
        for col in remainder: 
            X_train = X[selected_variables+[col]]
            X_train = sm.add_constant(X_train)
            model = sm.OLS(y,X_train).fit(disp=0)
            pval[col] = model.pvalues[col]
    
        min_pval = pval.min()
        if min_pval < 0.05: 
            selected_variables.append(pval.idxmin())
            while len(selected_variables) > 0:
                selected_X = X[selected_variables]
                selected_X = sm.add_constant(selected_X)
                selected_pval = sm.OLS(y,selected_X).fit(disp=0).pvalues.iloc[1:]
                max_pval = selected_pval.max()
                if max_pval >= 0.05:   ## 최대 p-value값이 기준값보다 크거나 같으면 제외
                    remove_variable = selected_pval.idxmax()
                    selected_variables.remove(remove_variable)
                else:
                    break
            
            step += 1
            steps.append(step)
            adj_r_squared = sm.OLS(y,sm.add_constant(X[selected_variables])).fit(disp=0).rsquared_adj
            adjusted_r_squared.append(adj_r_squared)
            sv_per_step.append(selected_variables.copy())
        else:
            break
    return selected_variables

## LRC

In [ ]:
def LRC(model):   
    coef = model.params
    LRC_table = pd.DataFrame({'feature_name': coef.index, 'LRC': np.abs(coef.values)/np.sum(np.abs(coef.values))})
    LRC_table['rank'] = LRC_table['LRC'].rank(method='average', ascending=True)
    return LRC_table

In [ ]:
feature_bank = [i for i in bank.columns if i not in ['date']]
feature_card = [i for i in card.columns if i not in ['date']]
feature_stock = [i for i in stock.columns if i not in ['date']]
feature_life = [i for i in life.columns if i not in ['date']]

def Weight(LRC_table):
    for i in [('은행', feature_bank), ('카드', feature_card), ('투자', feature_stock),('라이프',feature_life)]:
        print(f'{i[0]} variable:', LRC_table.query('feature_name in @i[1]')['feature_name'].values)
        print(f'{i[0]}:', (LRC_table.query('feature_name in @i[1]')['rank'].sum()/sum(range(1,LRC_table.shape[0]+1))).round(3))

In [ ]:
# X는 독립변수, y는 종속변수입니다.
X = df.iloc[:,1:]
y = y_train

## RandomForestRegressor

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 학습
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 피처 중요도 추출
importances = model.feature_importances_

# 피처 중요도를 데이터프레임으로 변환
feature_importances = pd.DataFrame({'feature': X_train.columns, 'importance': importances})

# 중요도에 따라 피처를 정렬
feature_importances = feature_importances.sort_values(by='importance', ascending=False)


In [ ]:
feature_importances

In [ ]:
# 중요도가 특정 임계값보다 낮은 피처 제거
selected_features = feature_importances[feature_importances['importance'] > 0.02]['feature']
selected_features

In [ ]:
# 중요도가 특정 임계값보다 낮은 피처 제거
X_selected = X[selected_features]

In [ ]:
X_selected
#성별
#지역
#유동거래_체크카드비율 
#유동거래_보험료금액 
#대출_일반대출평잔
#대출_신용대출평잔
#수신_예금평잔

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("Root Mean Squared Error:", rmse)

## MultiLinearRegression

In [ ]:
df.date.unique()
# 2208 데이터로 2210 예측
# 2210데이터로 2212 예측
# 2302데이터로 2304 예측

In [ ]:

X = X.reset_index(drop=True)
y = y.reset_index(drop=True)


In [ ]:
# 선택된 변수를 사용하여 모델을 만듭니다.
selected_variables = stepwise_feature_selection(X, y)
formula = "{} ~ {}".format(y.name, ' + '.join(selected_variables))
model_GT = sm.OLS.from_formula(formula, data=pd.concat([df,y], axis=1)).fit()
model_GT.summary()

In [ ]:
# 모델 저장
with open('MultiLinearRegression_1006.pkl', 'wb') as f:
    pickle.dump(model_GT, f)


In [ ]:
# 분야별 가중치 산정
# 분야별 가중치를 산출한다.
Weight(LRC(model_GT))

In [ ]:
model_GT.summary()

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
# 테스트 데이터로 예측을 수행합니다.
predictions = model_GT.predict(X_test)

# 3. 예측된 값과 실제 값의 차이를 통해 모델의 성능을 평가합니다.

# RMSE 계산
rmse = np.sqrt(mean_squared_error(target, predictions))

# R-squared 계산
r2 = r2_score(target, predictions)

print(f'RMSE: {rmse}')
print(f'R-squared: {r2}')


In [ ]:
predictions.describe()

In [ ]:
y_train.describe()

In [ ]:
# 모델 요약 출력
print(model_GT.summary())

## Tabnet

In [ ]:
def TabNet_modeling(X, y):
    # Road model
    reg = TabNetRegressor(optimizer_fn=torch.optim.Adam,
                          optimizer_params=dict(lr=1e-2),
                          scheduler_params={"step_size":50,"gamma":0.9},
                          scheduler_fn=torch.optim.lr_scheduler.StepLR,
                          mask_type='sparsemax', verbose=0, seed=2023)  # "sparsemax", entmax
    
    # Split data 
    X_train = X[X['date'] < 2306].drop('date', axis=1)
    X_test = X[X['date'] == 2306].drop('date', axis=1)
    y_train = y[X['date'] < 2306]
    y_test = y[X['date'] == 2306]

    # Further split training data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

    # Tunning
    reg.fit(X_train=X_train.values, y_train=y_train.values,
            eval_set=[(X_train.values, y_train.values), (X_val.values, y_val.values)],
            eval_name=['train', 'valid'],
            eval_metric=['rmse'],
            max_epochs=1000 , patience=300,
            batch_size=1024, virtual_batch_size=128,
            num_workers=0,
            drop_last=False)
    
    # Predict
    preds = reg.predict(X_test.values)
    test_mse = mean_squared_error(y_pred=preds, y_true=y_test)   
    
    # Selection
    fimportance = pd.DataFrame({'feature': X_train.columns, 'importance':reg.feature_importances_})
    unuse = fimportance.query('importance==0').feature.values.tolist()
    if len(unuse) != 0:
        X = X.drop(unuse, axis=1)
        return X, reg.best_cost, test_mse, reg
    else:
        return None, 'Done!', 'Done!', None


In [ ]:
# X는 독립변수, y는 종속변수입니다.
X = df.drop([i for i in df.columns if 'lb08m' in i], axis=1)
y = df[[i for i in df.columns if 'lb08m' in i]]

In [ ]:
first_selected_feature, val_rmse, test_rmse, first_model = TabNet_modeling(X, y)
print(f"BEST VALID SCORE FOR DATA : {val_rmse}")
print(f"FINAL TEST SCORE FOR DATA : {test_rmse}")

In [ ]:
def plot_feature_importances(fimportance):
    plt.figure(figsize=(10, 6))
    sns.barplot(x='importance', y='feature', data=fimportance.sort_values(by='importance', ascending=False))
    plt.title('Feature Importances')
    plt.show()

In [ ]:
# 피처 중요도 확인 (만약 모델이 학습이 완료되었다면)
if reg is not None:
    fimportance = pd.DataFrame({'feature': first_selected_feature.columns, 'importance': first_model.feature_importances_})
    print(fimportance.sort_values(by='importance', ascending=False))  # 피처 중요도 출력
    
    # 피처 중요도 시각화
    plot_feature_importances(fimportance)